Run the first 2 code cells only, this would generate a dataset.csv in the current directory and add the label of sonrisa (smile) to all the frames of the video taken, you must re run this process each time you desire to add a new feature to the dataset

In [ ]:
!pip uninstall xgboost

In [1]:
!pip install xgboost==1.6.1

ERROR: Could not find a version that satisfies the requirement xgboost==0.9
ERROR: No matching distribution found for xgboost==0.9


In [1]:
import xgboost as xgb
xgb.__version__

'1.6.1'

In [17]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb


mp_face_mesh = mp.solutions.face_mesh

class Facemesh:
    def __init__(self, label, import_df=True):
        left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]
        left_eyebrow = [276, 283, 282, 295, 285, 300, 293, 334, 296, 336]
        lips = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 185, 40, 39, 37, 0, 267, 269, 270, 409, 78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 191, 80, 81, 82, 13, 312, 311, 310, 415]
        right_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 246, 161, 160, 159, 158, 157, 173]
        right_eyebrow = [46, 53, 52, 65, 55, 70, 63, 105, 66, 107]
        self.kps = left_eye + left_eyebrow + lips + right_eye + right_eyebrow
        mp_face_mesh = mp.solutions.face_mesh
        self.label = label
        self.import_df = import_df
        self.face_mesh = mp_face_mesh.FaceMesh(  max_num_faces=1,
                                            refine_landmarks=True,
                                            min_detection_confidence=0.5,
                                            min_tracking_confidence=0.5)
        self.load_xgboost_model() 
        self.class_names = ['boca_abierta','neutral', 'ojo_derecho', 'ojo_izquierdo', 'sonrisa']
        
    def load_xgboost_model(self):
        file_name = 'modelos/model_2.json'
        model = xgb.Booster()
        model.load_model(file_name)
        self.xgb_model = model
        
    def get_image_landmarks(self,image):
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        return results
            
    def insert_dataframe(self, points):
        df_name = 'dataset_01.csv'
        df = pd.DataFrame(points).transpose()
        df.to_csv(df_name,mode='a',header=not self.import_df, index = False ) 
        
                        
    def draw_image_kps(self,image,facial_landmarks):
        height, width, _ = image.shape
        points = []
        for i in range(len(facial_landmarks.landmark)):
            if i in self.kps :
                pt= facial_landmarks.landmark[i]
                points.append(pt.x)
                points.append(pt.y)
                x = int(pt.x * width)
                y = int(pt.y * height)
                cv2.circle(image, (x, y), 1, (0, 100, 255), -1)
        return points
    
    def show_image(self, image):
        cv2.imshow("Image", image)
        cv2.waitKey(1)
    
    def draw_and_insert_kps(self,image,facial_landmarks):
        points = self.draw_image_kps(image,facial_landmarks)
        points.append(self.label)
        cv2.flip(image, 1)
        self.show_image(image)
        self.insert_dataframe(points)
    
    def draw_and_predict_gesture(self,image,facial_landmarks):
        points = self.draw_image_kps(image,facial_landmarks)
        df = pd.DataFrame(points).transpose()
        prediction = self.predict_gesture(df)
        cv2.flip(image, 1)
        cv2.putText(image,prediction,(200,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))
        self.show_image(image)
        
    def predict_gesture(self,df):
        formatted_df = xgb.DMatrix(df)
        prediction = int(self.xgb_model.predict(formatted_df))
        return self.class_names[prediction]
        

In [19]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

face_mesh = Facemesh("ojo_izquierdo",import_df = True)
num_frames = 0
max_frames = 500
do_classification = True

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            if do_classification:
                face_mesh.draw_and_predict_gesture(image,face_landmarks)
            else:
                face_mesh.draw_and_insert_kps(image,face_landmarks)
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

face_mesh = Facemesh("ojo_izquierdo",import_df = True)
num_frames = 0
max_frames = 500

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            face_mesh.draw_image_kps(image,face_landmarks)
cv2.destroyAllWindows()
cap.release()

This is for getting all the points that we desire as the contour

In [3]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df

,Unnamed: 0,0,7,13,14,17,33,37,39,40,...,387,388,390,398,402,405,409,415,466,label
0,0.0,0.524771,0.472195,0.524074,0.524118,0.524266,0.469790,0.514229,0.504052,0.496528,...,0.563230,0.566700,0.564955,0.541406,0.539144,0.541877,0.552934,0.548931,0.568775,sonrisa
1,1.0,0.677439,0.586151,0.684651,0.684459,0.694251,0.584403,0.675642,0.677446,0.678936,...,0.574215,0.576326,0.582930,0.578896,0.682753,0.690420,0.678271,0.680739,0.577953,sonrisa
2,NaN,0.000000,7.000000,13.000000,14.000000,17.000000,33.000000,37.000000,39.000000,40.000000,...,387.000000,388.000000,390.000000,398.000000,402.000000,405.000000,409.000000,415.000000,466.000000,label
3,0.0,0.522667,0.472494,0.522497,0.522482,0.522313,0.470072,0.512364,0.502720,0.496089,...,0.564420,0.567975,0.566535,0.542418,0.538375,0.541396,0.553187,0.548987,0.570100,sonrisa
4,1.0,0.676059,0.586140,0.685241,0.684971,0.697718,0.584078,0.673952,0.675535,0.677193,...,0.573736,0.575647,0.582090,0.578318,0.683179,0.693237,0.677420,0.680575,0.577130,sonrisa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,0.0,0.489493,0.418754,0.489987,0.490109,0.490022,0.415088,0.477244,0.465332,0.457396,...,0.570004,0.575526,0.573304,0.531672,0.510412,0.515203,0.531063,0.524800,0.578488,neutro
1774,1.0,0.592383,0.431372,0.610819,0.611286,0.635471,0.426097,0.587606,0.593914,0.601461,...,0.419230,0.423404,0.436741,0.429842,0.612255,0.631938,0.613018,0.616248,0.426623,neutro
1775,NaN,0.000000,7.000000,13.000000,14.000000,17.000000,33.000000,37.000000,39.000000,40.000000,...,387.000000,388.000000,390.000000,398.000000,402.000000,405.000000,409.000000,415.000000,466.000000,label
1776,0.0,0.490287,0.419085,0.490634,0.490685,0.490486,0.415381,0.478176,0.466257,0.458290,...,0.570479,0.575912,0.573586,0.533126,0.510745,0.515446,0.531121,0.524867,0.578864,neutro


In [5]:

left_eye = [(263, 249), (249, 390), (390, 373), (373, 374),
                               (374, 380), (380, 381), (381, 382), (382, 362),
                               (263, 466), (466, 388), (388, 387), (387, 386),
                               (386, 385), (385, 384), (384, 398), (398, 362)]

FACEMESH_LIPS = [(61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
                           (17, 314), (314, 405), (405, 321), (321, 375),
                           (375, 291), (61, 185), (185, 40), (40, 39), (39, 37),
                           (37, 0), (0, 267),
                           (267, 269), (269, 270), (270, 409), (409, 291),
                           (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
                           (14, 317), (317, 402), (402, 318), (318, 324),
                           (324, 308), (78, 191), (191, 80), (80, 81), (81, 82),
                           (82, 13), (13, 312), (312, 311), (311, 310),
                           (310, 415), (415, 308)]
LEFT_EYEBROW = [(276, 283), (283, 282), (282, 295),
                                   (295, 285), (300, 293), (293, 334),
                                   (334, 296), (296, 336)]
RIGHT_EYE = [(33, 7), (7, 163), (163, 144), (144, 145),
                                (145, 153), (153, 154), (154, 155), (155, 133),
                                (33, 246), (246, 161), (161, 160), (160, 159),
                                (159, 158), (158, 157), (157, 173), (173, 133)]
RIGHT_EYEBROW = [(46, 53), (53, 52), (52, 65), (65, 55),
                                    (70, 63), (63, 105), (105, 66), (66, 107)]
a = []
for i in left_eye:
    if i[0] not in a:
        a.append(i[0])
    if  i[1] not in a:
        a.append(i[1])
print(a)

[263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]


This fragments of code should not be runned just examples and past code

In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()
 
center_ids = [468, 473]

while True:
    # Image
    
    ret, image = cap.read()
    if ret is not True:
        break
    
    height, width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_image)
    
    if result.multi_face_landmarks != None:

        for facial_landmarks in result.multi_face_landmarks:
            for i in range(len(facial_landmarks.landmark)):
                pt1 = facial_landmarks.landmark[i]
                x = int(pt1.x * width)
                y = int(pt1.y * height)
                cv2.circle(image, (x, y), 1, (100, 100, 0), -1)
    
    cv2.imshow("Image", image)
    cv2.waitKey(1)

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 466, 388, 387, 386, 385, 384, 398]

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
